In [1]:
from keras.models import model_from_yaml
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
from tqdm import tqdm_notebook
from glob import glob
% matplotlib inline

Using TensorFlow backend.


In [2]:
with open("ModelArchive/u-net-model.yaml") as f:
    model = model_from_yaml(f.read())
model.load_weights("ModelArchive/unet_best_checkpoint.hdf5")

In [3]:
from utils.keras_tools import ImageDirectoryIterator, x_tif_reader, y_tif_reader, load_from_directories
import re, os

In [4]:
def save_tif(output, array):
    driver = gdal.GetDriverByName('GTiff')
    new_tif = driver.Create(output, 256, 256, 1, gdal.GDT_Byte)
    new_tif.GetRasterBand(1).WriteArray(array)

In [5]:
filepaths = glob("/root/notebooks/data/dest/110025_*.jpeg")

gen = ImageDirectoryIterator([(filepath, None) for filepath in filepaths], x_tif_reader, None, batch_size=16, shuffle=False)
predic_y = model.predict_generator(gen, steps=len(gen), verbose=1)
predic_y_byte = (predic_y*255).astype(np.uint8)
for i, filepath in enumerate(filepaths):
    filename = os.path.basename(filepath)
    gray = predic_y_byte[i].reshape(256, 256, 1)
    wb = np.dstack([gray,gray,gray])
    plt.imsave(os.path.join("/root/notebooks/data/pred", os.path.splitext(filename)[0]+".png"), wb)

总共找到233个样本。
15/15 [==============================] - 4s     
